- __*Method chaining* (encadeamento de métodos)__
	- __`.assign()`__
	- __`.pipe()`__
- `for` vs *List comprehension*
- Usando `.apply()`
- `.apply(axis = 0)` 
- Barra de progresso usando `tqdm`
- Pandas Profiling para exploração dos dados e para garantir a qualidade dos dados
- Mostrar o dataframe usando o `style`
- Copiar e colar no Excel `.to_clipboard()`
- Funções de janela móvel (*window functions*)
- Fechamento 2o projeto

In [92]:
import pandas as pd
import os
import numpy as np

# Tabela NBA janeiro 2021


In [93]:
path = 'input/nba_2021.csv'
os.makedirs('./input', exist_ok=True)

resultados = pd.read_html("https://www.basketball-reference.com/leagues/NBA_2021_games-january.html")[0]
resultados.to_csv(path)
df = resultados.copy()
resultados.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,LOG,Arena,Notes
0,"Fri, Jan 1, 2021",7:00p,Memphis Grizzlies,108,Charlotte Hornets,93,Box Score,NaN,0,2:04,Spectrum Center,NaN
1,"Fri, Jan 1, 2021",7:00p,Miami Heat,83,Dallas Mavericks,93,Box Score,NaN,0,2:12,American Airlines Center,NaN
2,"Fri, Jan 1, 2021",7:00p,Boston Celtics,93,Detroit Pistons,96,Box Score,NaN,0,2:04,Little Caesars Arena,NaN
3,"Fri, Jan 1, 2021",7:30p,Atlanta Hawks,114,Brooklyn Nets,96,Box Score,NaN,0,2:00,Barclays Center,NaN
4,"Fri, Jan 1, 2021",8:00p,Chicago Bulls,96,Milwaukee Bucks,126,Box Score,NaN,0,2:09,Fiserv Forum,NaN


In [94]:
colunas = {
    'Date': 'data',
    'Start (ET)': 'horario',
    'Visitor/Neutral': 'time_visitante',
    'PTS': 'pts_visit',
    'Home/Neutral': 'time_casa',
    'PTS.1': 'pts_casa',
    'Unnamed: 6': 'a',
    'Unnamed: 7': 'b',
    'Attend.': 'publico',
    'Notes': 'anot'
}
result_renomeado = resultados.rename(columns=colunas)

In [95]:
result_renomeado['data'] = pd.to_datetime(result_renomeado.data)

In [96]:
result_renomeado_data = result_renomeado.set_index('data')

In [97]:
result_renomeado_data_cols = result_renomeado_data.drop(['horario','a','b','publico','anot'], axis=1)

In [98]:
result_renomeado_data_cols['total'] = (result_renomeado_data_cols['pts_visit']
                                       + result_renomeado_data_cols['pts_casa'])

In [99]:
def det_vencedor(df):
    if df['pts_visit'] < df['pts_casa']:
        df['ganhador'] = 'casa'
    else:
        df['ganhador'] = 'visitante'
    return df

In [100]:
result_renomeado_data_cols = result_renomeado_data_cols.apply(det_vencedor, axis=1)

In [101]:
result_renomeado_data_cols.ganhador.value_counts(normalize=True)

ganhador
casa         0.531532
visitante    0.468468
Name: proportion, dtype: float64

In [102]:
# def det_vencedor(df):
#     df['ganhador'] = df['pts_casa'].gt(df['pts_visit']).replace({True: 'casa', False: 'visitante'})
#     return df

def det_vencedor(df):
    if df['pts_visit'] < df['pts_casa']:
        df['ganhador'] = 'casa'
    else:
        df['ganhador'] = 'visitante'
    return df

resultados = pd.read_html("https://www.basketball-reference.com/leagues/NBA_2021_games-january.html")[0]

colunas = {
    'Date': 'data',
    'Start (ET)': 'horario',
    'Visitor/Neutral': 'time_visitante',
    'PTS': 'pts_visit',
    'Home/Neutral': 'time_casa',
    'PTS.1': 'pts_casa',
    'Unnamed: 6': 'a',
    'Unnamed: 7': 'b',
    'Attend.': 'publico',
    'Notes': 'anot'
}
result_renomeado = resultados.rename(columns=colunas)
result_renomeado['data'] = pd.to_datetime(result_renomeado.data)
result_renomeado_data = result_renomeado.set_index('data')
result_renomeado_data_cols = result_renomeado_data.drop(['horario','a','b','publico','anot'], axis=1)
result_renomeado_data_cols['total'] = result_renomeado_data_cols['pts_visit']+result_renomeado_data_cols['pts_casa']
result_renomeado_data_cols_venc = result_renomeado_data_cols.apply(det_vencedor, axis=1)
result_renomeado_data_cols_venc

,time_visitante,pts_visit,time_casa,pts_casa,LOG,Arena,total,ganhador
data,,,,,,,,
2021-01-01,Memphis Grizzlies,108,Charlotte Hornets,93,2:04,Spectrum Center,201,visitante
2021-01-01,Miami Heat,83,Dallas Mavericks,93,2:12,American Airlines Center,176,casa
2021-01-01,Boston Celtics,93,Detroit Pistons,96,2:04,Little Caesars Arena,189,casa
2021-01-01,Atlanta Hawks,114,Brooklyn Nets,96,2:00,Barclays Center,210,visitante
2021-01-01,Chicago Bulls,96,Milwaukee Bucks,126,2:09,Fiserv Forum,222,casa
...,...,...,...,...,...,...,...,...
2021-01-31,Utah Jazz,117,Denver Nuggets,128,2:18,Ball Arena,245,casa
2021-01-31,Philadelphia 76ers,119,Indiana Pacers,110,2:19,Bankers Life Fieldhouse,229,visitante
2021-01-31,Orlando Magic,102,Toronto Raptors,115,2:14,Amalie Arena,217,casa


# *Method chaining* (encadeamento de métodos)


### Assign
`.assign()` faz a mesma coisa que o `=`, ou seja, 

`df['data'] = pd.to_datetime(df['data'])` é a mesma coisa que `df.assign(data = lambda x: pd.to_datetime(x.data))`

In [103]:
resultados = result_renomeado_data_cols_venc
resultados['total'] = resultados['pts_visit']+resultados['pts_casa']
resultados

,time_visitante,pts_visit,time_casa,pts_casa,LOG,Arena,total,ganhador
data,,,,,,,,
2021-01-01,Memphis Grizzlies,108,Charlotte Hornets,93,2:04,Spectrum Center,201,visitante
2021-01-01,Miami Heat,83,Dallas Mavericks,93,2:12,American Airlines Center,176,casa
2021-01-01,Boston Celtics,93,Detroit Pistons,96,2:04,Little Caesars Arena,189,casa
2021-01-01,Atlanta Hawks,114,Brooklyn Nets,96,2:00,Barclays Center,210,visitante
2021-01-01,Chicago Bulls,96,Milwaukee Bucks,126,2:09,Fiserv Forum,222,casa
...,...,...,...,...,...,...,...,...
2021-01-31,Utah Jazz,117,Denver Nuggets,128,2:18,Ball Arena,245,casa
2021-01-31,Philadelphia 76ers,119,Indiana Pacers,110,2:19,Bankers Life Fieldhouse,229,visitante
2021-01-31,Orlando Magic,102,Toronto Raptors,115,2:14,Amalie Arena,217,casa


In [104]:
resultados.assign(total_2 = lambda x: x['pts_visit'] + x['pts_casa'])

,time_visitante,pts_visit,time_casa,pts_casa,LOG,Arena,total,ganhador,total_2
data,,,,,,,,,
2021-01-01,Memphis Grizzlies,108,Charlotte Hornets,93,2:04,Spectrum Center,201,visitante,201
2021-01-01,Miami Heat,83,Dallas Mavericks,93,2:12,American Airlines Center,176,casa,176
2021-01-01,Boston Celtics,93,Detroit Pistons,96,2:04,Little Caesars Arena,189,casa,189
2021-01-01,Atlanta Hawks,114,Brooklyn Nets,96,2:00,Barclays Center,210,visitante,210
2021-01-01,Chicago Bulls,96,Milwaukee Bucks,126,2:09,Fiserv Forum,222,casa,222
...,...,...,...,...,...,...,...,...,...
2021-01-31,Utah Jazz,117,Denver Nuggets,128,2:18,Ball Arena,245,casa,245
2021-01-31,Philadelphia 76ers,119,Indiana Pacers,110,2:19,Bankers Life Fieldhouse,229,visitante,229
2021-01-31,Orlando Magic,102,Toronto Raptors,115,2:14,Amalie Arena,217,casa,217


## Pipe
Dá pra fazer bastante coisa com as funções internas do pandas/python, mas as vezes a gente precisa usar alguma função criada por nós mesmos, e para isso que serve o `.pipe()` e tem a funcionalidade parecida com o `%>%` (leia-se pipe) do R.
Onde o resultado da função a esquerda entra como primeiro parâmetro na função da direita.


#### Em R
``` R

sqrt(sum(x))

x %>% sum() %>% sqrt()

```


### Receita de bolo sem pipe. Tente entender o que é preciso fazer.

``` R

esfrie(
  asse(
    coloque(
      bata(
        acrescente(
          recipiente(
            rep("farinha", 2), 
            "água", 
            "fermento", 
            "leite", 
            "óleo"
          ), 
          "farinha", 
          ate = "macio"
        ), 
        duracao = "3min"
      ), 
      lugar = "forma", 
      tipo = "grande", 
      untada = TRUE
    ), 
    duracao = "50min"
  ), 
  lugar = "geladeira", 
  duracao = "20min"
)

```

### Veja como o código acima pode ser reescrito utilizando-se o pipe. Agora realmente se parece com uma receita de bolo.


``` R
recipiente(rep("farinha", 2), "água", "fermento", "leite", "óleo") %>%
  acrescente("farinha", ate = "macio") %>%
  bata(duracao = "3min") %>%
  coloque(lugar = "forma", tipo = "grande", untada = TRUE) %>%
  asse(duracao = "50min") %>%
  esfrie(lugar = "geladeira", duracao = "20min")

```

trecho extraído do livro [curso-r](https://livro.curso-r.com/6-1-o-operador-pipe.html)

In [105]:
resultados.head(1)

,time_visitante,pts_visit,time_casa,pts_casa,LOG,Arena,total,ganhador
data,,,,,,,,
2021-01-01,Memphis Grizzlies,108,Charlotte Hornets,93,2:04,Spectrum Center,201,visitante


In [106]:
resultados = resultados.drop('ganhador', axis=1)
resultados.head(1)

,time_visitante,pts_visit,time_casa,pts_casa,LOG,Arena,total
data,,,,,,,
2021-01-01,Memphis Grizzlies,108,Charlotte Hornets,93,2:04,Spectrum Center,201


In [107]:
resultados = resultados.apply(det_vencedor, axis=1)
resultados

,time_visitante,pts_visit,time_casa,pts_casa,LOG,Arena,total,ganhador
data,,,,,,,,
2021-01-01,Memphis Grizzlies,108,Charlotte Hornets,93,2:04,Spectrum Center,201,visitante
2021-01-01,Miami Heat,83,Dallas Mavericks,93,2:12,American Airlines Center,176,casa
2021-01-01,Boston Celtics,93,Detroit Pistons,96,2:04,Little Caesars Arena,189,casa
2021-01-01,Atlanta Hawks,114,Brooklyn Nets,96,2:00,Barclays Center,210,visitante
2021-01-01,Chicago Bulls,96,Milwaukee Bucks,126,2:09,Fiserv Forum,222,casa
...,...,...,...,...,...,...,...,...
2021-01-31,Utah Jazz,117,Denver Nuggets,128,2:18,Ball Arena,245,casa
2021-01-31,Philadelphia 76ers,119,Indiana Pacers,110,2:19,Bankers Life Fieldhouse,229,visitante
2021-01-31,Orlando Magic,102,Toronto Raptors,115,2:14,Amalie Arena,217,casa


In [108]:
def det_vencedor(df):
    df['ganhador'] = df['pts_casa'].gt(df['pts_visit']).replace({True: 'casa', False: 'visitante'})
    return df

In [109]:
resultados.pipe(det_vencedor)

,time_visitante,pts_visit,time_casa,pts_casa,LOG,Arena,total,ganhador
data,,,,,,,,
2021-01-01,Memphis Grizzlies,108,Charlotte Hornets,93,2:04,Spectrum Center,201,visitante
2021-01-01,Miami Heat,83,Dallas Mavericks,93,2:12,American Airlines Center,176,casa
2021-01-01,Boston Celtics,93,Detroit Pistons,96,2:04,Little Caesars Arena,189,casa
2021-01-01,Atlanta Hawks,114,Brooklyn Nets,96,2:00,Barclays Center,210,visitante
2021-01-01,Chicago Bulls,96,Milwaukee Bucks,126,2:09,Fiserv Forum,222,casa
...,...,...,...,...,...,...,...,...
2021-01-31,Utah Jazz,117,Denver Nuggets,128,2:18,Ball Arena,245,casa
2021-01-31,Philadelphia 76ers,119,Indiana Pacers,110,2:19,Bankers Life Fieldhouse,229,visitante
2021-01-31,Orlando Magic,102,Toronto Raptors,115,2:14,Amalie Arena,217,casa


In [110]:
def det_vencedor(df):
    df['ganhador'] = np.where(df['pts_visit']<df['pts_casa'], 'casa', 'visitante')
    return df

resultados = pd.read_html("https://www.basketball-reference.com/leagues/NBA_2021_games-january.html")[0]

colunas = {
    'Date': 'data',
    'Start (ET)': 'horario',
    'Visitor/Neutral': 'time_visitante',
    'PTS': 'pts_visit',
    'Home/Neutral': 'time_casa',
    'PTS.1': 'pts_casa',
    'Unnamed: 6': 'a',
    'Unnamed: 7': 'b',
    'Attend.': 'publico',
    'Notes': 'anot'
}
result_renomeado = resultados.rename(columns=colunas) #poderia ser resultados.columns = cols
result_renomeado['data'] = pd.to_datetime(result_renomeado.data)
result_renomeado_data = result_renomeado.set_index('data')
result_renomeado_data_cols = result_renomeado_data.drop(['horario','a','b','publico','anot'], axis=1)
result_renomeado_data_cols['total'] = result_renomeado_data_cols['pts_visit']+result_renomeado_data_cols['pts_casa']
result_renomeado_data_cols_venc = result_renomeado_data_cols.apply(det_vencedor, axis=1)
result_renomeado_data_cols_venc

,time_visitante,pts_visit,time_casa,pts_casa,LOG,Arena,total,ganhador
data,,,,,,,,
2021-01-01,Memphis Grizzlies,108,Charlotte Hornets,93,2:04,Spectrum Center,201,visitante
2021-01-01,Miami Heat,83,Dallas Mavericks,93,2:12,American Airlines Center,176,casa
2021-01-01,Boston Celtics,93,Detroit Pistons,96,2:04,Little Caesars Arena,189,casa
2021-01-01,Atlanta Hawks,114,Brooklyn Nets,96,2:00,Barclays Center,210,visitante
2021-01-01,Chicago Bulls,96,Milwaukee Bucks,126,2:09,Fiserv Forum,222,casa
...,...,...,...,...,...,...,...,...
2021-01-31,Utah Jazz,117,Denver Nuggets,128,2:18,Ball Arena,245,casa
2021-01-31,Philadelphia 76ers,119,Indiana Pacers,110,2:19,Bankers Life Fieldhouse,229,visitante
2021-01-31,Orlando Magic,102,Toronto Raptors,115,2:14,Amalie Arena,217,casa


Mostrar o de cima e falar novamente sobre as monte de vars e perguntar como podemos fazer usando pipe e assign

In [111]:
df = (df.rename(columns=colunas)
    .assign(data = lambda x: pd.to_datetime(x.data))
    .set_index('data')
    .drop(['horario','a','b','publico','anot'], axis=1)
    .assign(total = lambda x: x['pts_visit']+x['pts_casa'])
    .pipe(det_vencedor)
)
df.head()

,time_visitante,pts_visit,time_casa,pts_casa,LOG,Arena,total,ganhador
data,,,,,,,,
2021-01-01,Memphis Grizzlies,108,Charlotte Hornets,93,2:04,Spectrum Center,201,visitante
2021-01-01,Miami Heat,83,Dallas Mavericks,93,2:12,American Airlines Center,176,casa
2021-01-01,Boston Celtics,93,Detroit Pistons,96,2:04,Little Caesars Arena,189,casa
2021-01-01,Atlanta Hawks,114,Brooklyn Nets,96,2:00,Barclays Center,210,visitante
2021-01-01,Chicago Bulls,96,Milwaukee Bucks,126,2:09,Fiserv Forum,222,casa
